# Apache Iceberg デモンストレーション

このノートブックでは、Apache Icebergの主要な機能を実際のコードを通じて学習します。

## 学習内容
1. **Time Travel**: スナップショット機能による履歴管理と過去データの参照
2. **メタデータ管理**: テーブル履歴とスナップショットの管理
3. **スキーマエボリューション**: カラムの追加・変更・削除の安全な実行

## 前提条件
- Unity Catalogが有効化されている環境
- パーソナル領域（users.{username}）へのアクセス権限
- Iceberg形式のテーブル作成権限


## 1. Time Travel（タイムトラベル）

Apache Icebergの**Time Travel**機能は、テーブルのスナップショット履歴を管理し、過去の状態を参照・復旧できる機能です。

### 主な用途
- **データの監査**: 特定時点でのデータ状態の確認
- **誤操作からの復旧**: 間違ったデータ変更からの復元
- **A/Bテスト**: 異なる時点のデータでの比較分析
- **レポート再現**: 過去のレポート結果の再現

### Icebergのスナップショット機能
- **スナップショット**: 各操作でテーブルの状態を記録
- **メタデータ管理**: 効率的なメタデータ構造で高速アクセス
- **タイムスタンプ/スナップショットID**: 複数の方法で過去データにアクセス可能


In [0]:
-- Unity Catalogのパーソナル領域を使用
-- 各ユーザーは自分のスキーマ名に変更してください
USE CATALOG users;
USE SCHEMA yukiteru_koide;  -- ← ここを自分のユーザー名に変更


### ステップ1: デモ用Icebergテーブルの作成と初期データ投入

Apache Icebergテーブルを作成し、初期データを投入します。この操作により最初のスナップショットが作成されます。


In [0]:
-- Apache Icebergテーブルを作成（USING ICEBERGを指定）
CREATE OR REPLACE TABLE iceberg_demo (
  id       INT,           -- 主キー
  amount   DECIMAL(10,2), -- 金額
  country  STRING,        -- 国コード
  category STRING,        -- 商品カテゴリ
  ts       TIMESTAMP      -- タイムスタンプ
) USING ICEBERG;

-- 初期データを投入（最初のスナップショットが作成される）
INSERT INTO iceberg_demo VALUES
  (1, 100.00, 'JP', 'Electronics', current_timestamp()),
  (2, 200.00, 'US', 'Books', current_timestamp()),
  (3, 150.00, 'JP', 'Clothing', current_timestamp()),
  (4, 300.00, 'UK', 'Electronics', current_timestamp());

-- 投入されたデータを確認
SELECT * FROM iceberg_demo ORDER BY id;


### ステップ2: 追加データの投入（新しいスナップショット作成）

新しいデータを追加してスナップショットを増やします。Icebergでは各操作が新しいスナップショットとして記録されます。


In [0]:
-- 新しいデータを追加してスナップショットを増やす
INSERT INTO iceberg_demo VALUES
  (5, 250.00, 'DE', 'Sports', current_timestamp()),
  (6, 180.00, 'FR', 'Books', current_timestamp());

-- 追加後のデータを確認
SELECT * FROM iceberg_demo ORDER BY id;


### ステップ3: 履歴情報の確認

まず、テーブルの操作履歴とスナップショット情報を確認します。Icebergでは`DESCRIBE HISTORY`コマンドで詳細な履歴を参照できます。


In [0]:
-- テーブルの履歴を確認
DESCRIBE HISTORY iceberg_demo;

### ステップ4: データ更新操作（さらなるスナップショット作成）

既存データを更新して履歴を作成します。Icebergでは更新操作も新しいスナップショットとして記録されます。


In [0]:
-- 既存データを更新して履歴を作成
UPDATE iceberg_demo 
SET amount = amount * 1.1 
WHERE country = 'JP';

-- 更新後のデータを確認（日本のデータが10%増加）
SELECT * FROM iceberg_demo WHERE country = 'JP' ORDER BY id;


In [0]:
-- テーブルの履歴を確認
DESCRIBE HISTORY iceberg_demo;

### ステップ5: 誤操作のシミュレーション

**⚠️ 注意**: これは意図的な誤操作です！

Electronics カテゴリのデータを間違って削除してしまう状況を再現します。実際の運用では避けるべき操作ですが、復旧デモのために実行します。


In [0]:
-- Electronics カテゴリのデータを誤って削除
DELETE FROM iceberg_demo WHERE category = 'Electronics';

-- 削除後の状態を確認（Electronicsカテゴリが消えていることを確認）
SELECT * FROM iceberg_demo ORDER BY id;


In [0]:
-- テーブル履歴を確認
DESCRIBE HISTORY iceberg_demo;


### ステップ6: タイムスタンプによるTime Travel

特定の時刻のデータを参照します（削除前の状態）。

**補足**: 現在時刻から相対的に指定することも可能
```sql
SELECT * FROM iceberg_demo FOR SYSTEM_TIME AS OF (current_timestamp() - INTERVAL 10 MINUTES);
```

※ 実際の時刻は上記の`DESCRIBE HISTORY`の結果を参考に調整してください


In [0]:
-- 特定の時刻のデータを参照
SELECT * FROM iceberg_demo 
FOR SYSTEM_TIME AS OF '2025-10-02T06:28:48.888+00:00'
ORDER BY id;


### ステップ7: データの復旧（INSERT INTO ... SELECT）
- 【復旧方法の特徴】
- 部分的な復旧が可能
- 条件を指定して特定データのみ復元
- 複数のスナップショットからのデータ統合も可能

In [0]:
-- Time Travelを使用して削除されたデータを復旧
-- 削除前のスナップショットから削除されたレコードを特定して再挿入
INSERT INTO iceberg_demo
SELECT * FROM iceberg_demo 
FOR SYSTEM_TIME AS OF '2025-10-02T06:28:48.888+00:00'
WHERE category = 'Electronics';

-- 復旧後のデータを確認
SELECT * FROM iceberg_demo ORDER BY id;
